In [ ]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from pandas.plotting import table
from datetime import datetime, date, timedelta
import time
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML, Markdown
import plotly.figure_factory as ff
import os


In [ ]:
# Calculate the week-ending date for last week
today = date.today()
idx = (today.weekday()+1)%7
week_ending = today - timedelta(7+idx-7)
#week_ending

start_date = week_ending-timedelta(days=6)

<h1><center>Labor & Equipment Weekly Report</center></h1>
<br>
<h3><center>Week Ending {{week_ending}}</center></h3>  

In [ ]:
Markdown("Week Ending {}".format(week_ending))

In [ ]:
headers = {'Content-Type':'application/x-www-form-urlencoded',
           'Cache-Control': 'no-cache',}
domain = "https://api.hcssapps.com/"
token_expiration = time.time()-3601
auth_header = {}

MyBusinessUnitId = os.environ.get('FCC_HCSS_BUSSINESS_UNIT_ID')

In [ ]:
#Get token

def get_token():
    global auth_header
    client_id = os.environ.get('HCSS_API_ID') #insert your HCSS API client ID issued by HCSS
    client_secret = os.environ.get('HCSS_API_SECRET') #insert your HCSS API client secret issued by HCSS
    scope = 'heavyjob:read dis:read timecards:read'
    grant_type = 'client_credentials'

    token_url = domain + "identity/connect/token"
    payload = {'client_id': client_id,
              'client_secret': client_secret,
              'scope' :scope,
              'grant_type' : grant_type
              }

    try:
        #conn.request("POST", token_url, payload, headers)
        response = requests.post(token_url, data=payload, headers=headers)
        #print(response.text)

        #response = conn.getresponse()
        #data = response.read()
        #print(data.decode("utf-8"))
        #conn.close
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        

    token = json.loads(response.text)

    #validate token
    auth_header = {'Authorization': token.get('token_type') + " " + token.get('access_token')}
    token_expiration = time.time()+token.get('expires_in')
    #print(auth_header)
    
    return auth_header, token_expiration

#check for current token
def check_token():
    global auth_header
    global token_expiration
    #print(token_expiration)
    if time.time()<token_expiration:
        token_status = True
        #print('token still good')
    else:
        try:
            auth_header, token_expiration = get_token()
            token_status = True
            #print('token expired, had to re-authorize')
        except:
            token_status = False
            #print('token expired')
    return token_status
        

#get business units

def get_business_units():
    check_token()
    bu_url = domain + 'heavyjob/api/v1/businessUnits'
    #print(bu_url)

    business_units = requests.get(bu_url, headers=auth_header)

    #print(business_units.text)

    df_business_units = pd.read_json(business_units.text)

    del df_business_units['id']
    del df_business_units['credentialsId']
    return df_business_units

#get job list

def get_job_list(bus_unit):
    check_token()
    jobs_url = domain + 'heavyjob/api/v1/jobs?businessUnitId=' + bus_unit
    #print(bu_url)

    jobs = requests.get(jobs_url, headers=auth_header)

    #print(business_units.text)

    df = pd.read_json(jobs.text)

    #del df_business_units['id']
    #del df_business_units['credentialsId']
    return df

#get project number jobId
def get_project_id(FCC_Proj_num):
    if FCC_Proj_num == 'ALL':
        return 'ALL'
    else:
        job = jobs.loc[jobs['code'] == FCC_Proj_num]
        return job.iloc[0]['id']
    


#get cost code tag list
def get_cost_code_tags(bus_unit):
    check_token()
    cost_code_tags = domain + 'heavyjob/api/v1/costCodeSetupFilters?businessUnitId=' + bus_unit
    
    cost_codes = requests.get(cost_code_tags, headers=auth_header)

    df = pd.read_json(cost_codes.text)

    return df

#get code code tag relationship list
def get_cost_code_tags_relations():
    check_token()
    try:
        cost_code_tags_relations = domain + 'heavyjob/api/v1/costCodeTags?jobId=961a7f32-b798-4b42-ab63-83aa73db341d'

        cost_codes_relations = requests.get(cost_code_tags_relations, headers=auth_header)

        
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        
   
    return cost_codes_relations

#get list of cost adjustments
def get_cost_adjustments(bus_unit):
    check_token()
    cost_adjustments = domain + 'heavyjob/api/v1/businessUnits/'+bus_unit+'/costAdjustments'
    
    data = requests.get(cost_adjustments, headers=auth_header)

    #print(business_units.text)

    df = pd.read_json(data.text)

    #del df_business_units['id']
    #del df_business_units['credentialsId']
    return df

#get job cost
def get_job_cost(jobId):
    check_token()
    job_cost = domain + 'heavyjob/api/v1/jobs/'+jobId+'/costs'
    
    data = requests.get(job_cost, headers=auth_header).json()
    #num_pages = data['last_page']

    #print(type(data))

    #df = json_normalize(data.get('results'))

    #del df_business_units['id']
    #del df_business_units['credentialsId']
    return data

#get cost codes
def get_cost_codes(jobId):
    check_token()
    cost_codes = domain + 'heavyjob/api/v1/costCodes?jobId='+jobId
    
    data = requests.get(cost_codes, headers=auth_header).json()
    #num_pages = data['last_page']

    #print(type(data))

    df = json_normalize(data.get('results'))

    #del df_business_units['id']
    #del df_business_units['credentialsId']
    return df

#get timecard summary info
def get_timecard_summary(jobId, startDate=date(2021,1,1), endDate=date.today()):
    check_token()
    if jobId == 'ALL':
        request = domain + 'heavyjob/api/v1/timeCardInfo?startDate='+str(startDate)+'&endDate='+str(endDate)
    else:
        request = domain + 'heavyjob/api/v1/timeCardInfo?jobId='+jobId+'&startDate='+str(startDate)+'&endDate='+str(endDate)
    
    data = requests.get(request, headers=auth_header).json()
    
    df = json_normalize(data.get('results'))

    return df

#get timecard info
def get_timecard(timecard_id):
    check_token()
    request = domain + 'heavyjob/api/v1/timeCards/'+timecard_id
    
    data = requests.get(request, headers=auth_header).json()
    
    df = json_normalize(data)

    return df

#get employees
def get_employees(bus_unit):
    check_token()
    request = domain + 'heavyjob/api/v1/businessUnits/'+bus_unit+'/employees'
    
    data = requests.get(request, headers=auth_header).json()
    
    df = json_normalize(data)

    return df

In [ ]:
business_units = get_business_units()
#business_units

In [ ]:
jobs = get_job_list(MyBusinessUnitId)
#jobs

In [ ]:
active_jobs = jobs[jobs.status == "active"]
active_jobs.head()

In [ ]:

fig = ff.create_table(active_jobs[["code","description", "status"]])
fig.update_layout(autosize=True)#False, width=500, height=300)
fig.write_image("H:/My Drive/Active Jobs in FCCRI.png", scale = 2, engine="kaleido")
fig.show()

In [ ]:
# cost_cost_tags = get_cost_code_tags(MyBusinessUnitId)
# cost_cost_tags

# tags = get_cost_code_tags_relations()
# tags.text

# cost_adjustments = get_cost_adjustments(MyBusinessUnitId)
# cost_adjustments

# cost_codes = get_cost_codes(get_project_id('7028'))
# cost_codes#.get('results')

job_cost = get_job_cost(get_project_id('7028'))
job_cost.get('results')


In [ ]:
cost_code_cost = pd.DataFrame(job_cost['costCodeCosts'])
cost_code_cost

In [ ]:
fccri_timecard_summary = timecard_summary[timecard_summary['businessUnitId'] == MyBusinessUnitId]
fccri_timecard_summary = fccri_timecard_summary.reset_index(drop=True)
#fccri_timecard_summary

In [ ]:
df_timecards_data = pd.DataFrame()
for timecard in range(len(fccri_timecard_summary)):
    data = (get_timecard(fccri_timecard_summary.id[timecard]))
    #data = data.drop(data.index[[0]])
    df_timecards_data = df_timecards_data.append(data, ignore_index=True)
#df_timecards_data

In [ ]:
for cost_code in range(len(df_timecards_data.costCodes[0])):
    print(df_timecards_data.costCodes[0][cost_code])

In [ ]:
#df = pd.DataFrame()
df = pd.DataFrame((df_timecards_data.iloc[[6]]['employees']))
df2 = pd.DataFrame(df.employees.iloc[0])#['employeeDescription']
df3 = pd.DataFrame(df2.iloc[[0]]['regularHours'])
df3.iloc[[0]]['regularHours']

In [ ]:
df_overtime = pd.DataFrame()
for timecard in range(len(fccri_timecard_summary)):
#    print("TIMECARD index: ",timecard, " ", df_timecards_data.id[timecard])
    for employee in range(len(df_timecards_data['employees'][timecard])):
        employee_name = df_timecards_data['employees'][timecard][employee]['employeeDescription']
#         print(employee_name,  #log to new df
#               df_timecards_data['employees'][timecard][employee]['regularHours'],
#               df_timecards_data['employees'][timecard][employee]['overtimeHours']
#             )
        for costcode in range(len(df_timecards_data['employees'][timecard][employee]['overtimeHours'])):
            ot_hours = 0
            #print("   Costcode index: ",costcode, " of ", len(df_timecards_data['employees'][timecard][employee]['overtimeHours']))
            ot_hours = df_timecards_data['employees'][timecard][employee]['overtimeHours'][costcode]['hours']
            #print(" OT Hours", ot_hours) #log to new df
#            print(df_timecards_data['employees'][timecard][employee]['overtimeHours'][costcode])
            #print(df_timecards_data.employees[0][employee]['employeeDescription'],df_timecards_data.employees[employee][0]['overtimeHours'][0]['hours'])
            data = {'Employee_Name':employee_name,'Overtime_Hours':ot_hours}
#            print(data)
            df_overtime = df_overtime.append(data, ignore_index=True)
df_overtime = df_overtime.groupby(['Employee_Name']).sum()
df_overtime = df_overtime.reset_index()
#df_overtime

In [ ]:
plot_df = df_overtime.sort_values(['Overtime_Hours']).reset_index(drop=True)
sns.barplot(data=plot_df, x='Overtime_Hours', y='Employee_Name', ci=None).set_title("Time Reporting for " + str(week_ending))
plt.rcParams['figure.figsize']=(20,40)
plt.axvline(x=9)
plt.savefig("H:/My Drive/Overtime Reporting.png")
plt.show()

In [ ]:
# f_timecards_data.iloc[24]['employees']#[0]['overtimeHours']
#test = df_timecards_data[df_timecards_data['foremanCode'] == '081DE']
#employees = test['employees'][0][0]['employeeDescription']
test = df_timecards_data[df_timecards_data['foremanCode'] == '081DE']

In [ ]:
test.employees.iloc[[0]].iloc[0]

In [ ]:
df_employee_OThours = json_normalize(df_timecards_data.employees[7])#,['overtimeHours','employeeCode', 'employeeDescription'])
df_employee_OThours

In [ ]:
employees = get_employees(MyBusinessUnitId)

In [ ]:
employees

In [ ]:
os.system('jupyter nbconvert --no-input --to html "C:/Users/apratt/HCSS API.ipynb"')

In [ ]:
get_cost_codes(get_project_id("7028"))